In [57]:
from requests import request
from bs4 import BeautifulSoup

from math import log2
import pandas as pd
import re
from nltk.tokenize import sent_tokenize
from konlpy.tag import Hannanum, Komoran

ma = Komoran()
from collections import defaultdict, Counter
from random import randint, choices


In [58]:
# dom = BeautifulSoup(request('get', 'https://www.goodchoice.kr/product/search/5').text, 'html5lib')
# ids = [a.attrs['data-ano'] for a in dom.select('li.list_2 > a[data-ano]')]

In [59]:
# url = 'https://www.goodchioce.kr/product/get_review_non'
# reviews = pd.DataFrame(columns=['ano', 'aepreg', 'rating', 'review'])
#
# for i in ids:
#     page = 1
#
#     while True:
#         resp = request('post', url, data={'page':page, 'ano': ids[0]})
#         for item in resp.json()['result']['items']:
#             reviews.loc[len(reviews)] = (i, item['aepreg'], item['epilrate'], item['aepcont'])
#         if page > resp.json()['total_page']:
#             break
#         page += 1
# resp.json
# len(resp.json()['result']['items'])

어휘기반 감성분석
-> 어휘(사전적 정의와 다름, 사람이 일일히 정한 것, 코퍼스 - Semi-supervised)
eg. 여기어때 -> 리뷰
1. 리뷰(평점) -> 두 그룹(<5, >5)  -> term ㅌ positive, negetive <- PMI
2.               X   대표단어 셋 -> term ㅌ 대표단어(positive), 대표단어(negetive) <- PMI
3.          -> Topic Model
4.                              -> 개별 토픽들이 어떤 극성(감성)
PMI, PPMI, NPMI + LDA(np, list<-)

In [60]:
with open('reviews.jpg', encoding='utf-8') as fp:
    reviews = pd.read_csv(fp)

reviews[reviews['rating'] < 10].describe()

,Unnamed: 0,ano,aepreg,rating
count,2138.000000,2138.000000,2.138000e+03,2138.000000
mean,3672.119738,56529.226848,1.611976e+09,6.657343
std,2053.632998,11729.379197,3.977837e+07,2.602609
min,0.000000,10681.000000,1.488162e+09,1.000000
25%,2053.750000,47204.000000,1.584905e+09,5.300000
50%,3764.000000,57279.000000,1.617594e+09,7.600000
75%,5502.500000,64461.000000,1.644820e+09,8.600000
max,7035.000000,72146.000000,1.668556e+09,9.600000


In [61]:
def cleanning1(org):
    text = re.sub(r'[^0-9A-Za-zㄱ-ㅎ가-힣.,!? ]+', ' ', org)
    result = list()
    for s in sent_tokenize(text):
        result.append(' '.join([t for t in ma.morphs(s) if len(t) > 1]))
    return ('\n'.join(result)).strip()

def cleanning2(org):
    text = re.sub(r'[^0-9A-Za-zㄱ-ㅎ가-힣.,!? ]+', ' ', org)
    result = list()
    for s in sent_tokenize(text):
        result.append(' '.join([t[0] for t in ma.pos(s) if len(t[0]) > 1 and t[1].startswith('V')]))
    return ('\n'.join(result)).strip()

def cleanning3(org):
    text = re.sub(r'[^0-9A-Za-zㄱ-ㅎ가-힣.,!? ]+', ' ', org)
    result = list()
    for s in sent_tokenize(text):
        result.append(' '.join([t for t in ma.nouns(s) if len(t) > 1]))
    return ('\n'.join(result)).strip()


In [62]:
reviews['clean1'] = reviews['review'].apply(cleanning1)
reviews['clean2'] = reviews['review'].apply(cleanning2)
reviews['clean3'] = reviews['review'].apply(cleanning3)

clean_reviews = reviews[reviews['rating'] < 10]

In [63]:
# 연구자 설정
negative = reviews[reviews['rating'] < 7]
positive = reviews[reviews['rating'] >= 7]

In [64]:
# pTDM = defaultdict(Counter)
# nTDM = defaultdict(Counter)

In [65]:
# for i, review in enumerate(negative['clean1']):
#     for t in review.split():
#         nTDM[t][i] += 1
#
# for i, review in enumerate(positive['clean1']):
#     for t in review.split():
#         pTDM[t][i] += 1

In [66]:
# V = list(set(list(nTDM.keys()) + list(pTDM.keys())))

In [67]:
# sorted({t: sum(tinfo.values()) for t, tinfo in nTDM.items()}.items(), key=lambda v:v[1], reverse=True)[:5]

In [68]:
# pProb = dict()
# length = len(reviews)
# for t in V:
#     if len(pTDM[t]) > 0:
#         pProb[t] = (len((pTDM[t]))+1)/(length+len(V))
#     # print(t, len(pTDM[t]))
#
# nProb = dict()
# for t in V:
#     if len(nTDM[t]) > 0:
#         nProb[t] = (len((nTDM[t]))+1)/(length+len(V))

In [69]:
# pJoint = dict()
# for t in V:
#     pJoint[t] = ((len(
#         set(list(pTDM[t].keys())).intersection(
#         set(list(nTDM[t].keys()))))+1) / (length+len(V)))

P(X,Y) = pJoint
P(X) = pProb
P(Y) = nProb
log2 P(X,Y) / (P(X)*P(Y))

In [70]:
# pWords = ['멋있고', '행복한']
# nWords = ['화장실', '시설']
#
# pWordsProb = [(len(pTDM[t])+len(nTDM[t]))/length for t in pWords]
# nWordsProb = [(len(pTDM[t])+len(nTDM[t]))/length for t in nWords]

In [71]:
# pPMI = defaultdict(float)
# for t in V:
#     pList = set(pTDM[t].keys())
#     nList = set(nTDM[t].keys())
#     for seed in pWords:
#         sList = set(list(pList[t].keys()) + list(nTDM[t].keys()))
#         cofreq = len(pList.intersection(nList).intersection(sList))
#         jp = (cofreq+1)/(length+len(V))
#         pp = (len(pList)+1)/(length+len(V))
#         np = (len(nList)+1)/(length+len(V))
#         pPMI[t] += log2(jp/(pp*np))
#
# nPMI = defaultdict(float)
# for t in V:
#     pList = set(pTDM[t].keys())
#     nList = set(nTDM[t].keys())
#     for seed in nWords:
#         sList = set(list(pList[t].keys()) + list(nTDM[t].keys()))
#         cofreq = len(pList.intersection(nList).intersection(sList))
#         jp = (cofreq+1)/(length+len(V))
#         pp = (len(pList)+1)/(length+len(V))
#         np = (len(nList)+1)/(length+len(V))
#      nPMI[t] += log2(jp/(pp*np))

In [72]:
TDM = defaultdict(Counter)
pTDM = defaultdict(Counter)
nTDM = defaultdict(Counter)

In [73]:
for row in reviews[['aepreg', 'clean1', 'rating']].itertuples():
    rid = row[1]
    review = row[2]
    rating = row[3]

    for t in review.split():
        TDM[t][rid] += 1
        if rating > 7:
            pTDM[t][rid] += 1
        else:
            nTDM[t][rid] += 1


In [74]:
pPMI = defaultdict()
nPMI = defaultdict()
pPPMI = defaultdict()
nPPMI = defaultdict()
pNPMI = defaultdict()
nNPMI = defaultdict()

In [75]:
V = len(TDM)
N = len(clean_reviews)
pN = len(positive)
nN = len(negative)

In [76]:
pProb = pN/N
nProb = nN/N

In [77]:
for t, docList in TDM.items():
    tProb = len(docList)/N
    tPJ = (len(pTDM[t])+1)/(N+V)
    tNJ = (len(nTDM[t])+1)/(N+V)
    pPMI[t] = log2(tPJ/(tProb*pProb))
    nPMI[t] = log2(tNJ/(tProb*nProb))
    pPPMI[t] = max([0, pPMI[t]])
    nPPMI[t] = max([0, nPMI[t]])
    pNPMI[t] = pPMI[t]/-log2(tPJ)
    nNPMI[t] = nPMI[t]/-log2(tNJ)


In [78]:
pWords = ['편하', '재밌', '즐겁', '예쁘'] # <- 주관적
nWords = ['찢어지', '아깝', '따지', '더럽']

In [96]:
pPMI = defaultdict(Counter)
nPMI = defaultdict(Counter)
pPPMI = defaultdict(Counter)
nPPMI = defaultdict(Counter)
pNPMI = defaultdict(Counter)
nNPMI = defaultdict(Counter)

for t, docList in TDM.items():
    tP = len(docList)/N

    for s in pWords:
        sDocList = set(TDM[s])
        sP = len(sDocList)/N
        tsJP = (len(sDocList.intersection(set(docList)))+1)/(N+V)
        pPMI[t][s] = log2(tsJP/(tP*sP))
        pPPMI[t][s] = max([0, pPMI[t][s]])
        pNPMI[t][s] = pPMI[t][s]/-log2(tsJP)

    for s in nWords:
        sDocList = set(TDM[s])
        sP = len(sDocList)/N
        tsJP = (len(sDocList.intersection(set(docList)))+1)/(N+V)
        nPMI[t][s] = log2(tsJP/(tP*sP))
        nPPMI[t][s] = max([0, nPMI[t][s]])
        nNPMI[t][s] = nPMI[t][s]/-log2(tsJP)

In [97]:
SO = {t:sum(pPMI[t].values())-sum(nPMI[t].values()) for t in TDM}
list(zip(sorted(SO.items(), key=lambda t:t[1], reverse=True)[:10],
sorted(SO.items(), key=lambda t:t[1], reverse=False)[:10]))
SO = {t:sum(pPPMI[t].values())-sum(nPPMI[t].values()) for t in TDM}
list(zip(sorted(SO.items(), key=lambda t:t[1], reverse=True)[:10],
sorted(SO.items(), key=lambda t:t[1], reverse=False)[:10]))
SO = {t:sum(pNPMI[t].values())-sum(nNPMI[t].values()) for t in TDM}
list(zip(sorted(SO.items(), key=lambda t:t[1], reverse=True)[:20],
sorted(SO.items(), key=lambda t:t[1], reverse=False)[:20]))

[(('재밌', -0.011060110956246483), ('더럽', -1.6527387821251556)),
 (('예쁘', -0.06732711818626225), ('따지', -1.6492447118585956)),
 (('즐겁', -0.14852721336879293), ('아깝', -1.5953313275859617)),
 (('편하', -0.16918800149363955), ('찢어지', -1.5221047884601107)),
 (('꾸미', -0.20579455782952727), ('장사', -1.4314528812769434)),
 (('오로라', -0.30458815462175837), ('최악', -1.4297607886261596)),
 (('구비', -0.30904620602471344), ('어이없', -1.4172900150318632)),
 (('조명', -0.328357803456081), ('반개', -1.4092200538651063)),
 (('여름', -0.3558305934449564), ('고장', -1.4056650022083883)),
 (('귀엽', -0.3613197239302297), ('잔소리', -1.3883093287866322)),
 (('맛있', -0.36807471531475056), ('신고', -1.3872995992063224)),
 (('시원', -0.36847954788281756), ('보증금', -1.3759782956523148)),
 (('행복', -0.37225547043914675), ('잖아요', -1.3752141464740142)),
 (('ㄹ게요', -0.37410299162617644), ('조차', -1.356445213187738)),
 (('감성', -0.3848963195826445), ('모양새', -1.3479222396049653)),
 (('추억', -0.38803741826388816), ('근성', -1.3479222396049653)),
 (('카

In [98]:
SO = {t:sum(pPMI[t].values())-sum(nPMI[t].values()) for t in TDM}
list(zip(sorted(SO.items(), key=lambda t:t[1], reverse=True)[:10],
sorted(SO.items(), key=lambda t:t[1], reverse=False)[:10]))
SO = {t:sum(pPPMI[t].values())-sum(nPPMI[t].values()) for t in TDM}
list(zip(sorted(SO.items(), key=lambda t:t[1], reverse=True)[:10],
sorted(SO.items(), key=lambda t:t[1], reverse=False)[:10]))
SO = {t:sum(pNPMI[t].values())-sum(nNPMI[t].values()) for t in TDM}
list(zip(sorted(SO.items(), key=lambda t:t[1], reverse=True)[:20],
sorted(SO.items(), key=lambda t:t[1], reverse=False)[:20]))

[(('재밌', -0.011060110956246483), ('더럽', -1.6527387821251556)),
 (('예쁘', -0.06732711818626225), ('따지', -1.6492447118585956)),
 (('즐겁', -0.14852721336879293), ('아깝', -1.5953313275859617)),
 (('편하', -0.16918800149363955), ('찢어지', -1.5221047884601107)),
 (('꾸미', -0.20579455782952727), ('장사', -1.4314528812769434)),
 (('오로라', -0.30458815462175837), ('최악', -1.4297607886261596)),
 (('구비', -0.30904620602471344), ('어이없', -1.4172900150318632)),
 (('조명', -0.328357803456081), ('반개', -1.4092200538651063)),
 (('여름', -0.3558305934449564), ('고장', -1.4056650022083883)),
 (('귀엽', -0.3613197239302297), ('잔소리', -1.3883093287866322)),
 (('맛있', -0.36807471531475056), ('신고', -1.3872995992063224)),
 (('시원', -0.36847954788281756), ('보증금', -1.3759782956523148)),
 (('행복', -0.37225547043914675), ('잖아요', -1.3752141464740142)),
 (('ㄹ게요', -0.37410299162617644), ('조차', -1.356445213187738)),
 (('감성', -0.3848963195826445), ('모양새', -1.3479222396049653)),
 (('추억', -0.38803741826388816), ('근성', -1.3479222396049653)),
 (('카

In [ ]:
sumSO = 0.0
for t in ma.morphs('''22년 9월 8일 / 어른 5명, 초등저학년 3명, 20개월유아 1명
총 9명이 카라반 2개를 예약해서 사용했습니다
카라반 1개당 2인기준으로 추가비용 5명을 지불하고 입실

1. 제일 실망한 온수풀 온수가아닌 30도 미적지근한 더러운물
여러번 말했지만 소용없음 순환식 정수스타일 냄새도 나고
리뷰보고 예약했는데 최악입니다.
카라반 2동을 예약해서 온수1개만 사용한다고 7만원에 추가로 3만원더요구 함. 아니면 풀2개사용해서 14만원 내라고 함
돈내고 온수가아닌 정수 물 꼭 농수로 같았어요

2. 어찌저찌 2동에 인원추가비, 불멍 장작비, 숯불비용 추가비 현금 밎 계좌이체 요구 카드 안된다 현금영수증 안된다 (세무서 또는 국세청 신고예정)

3. 불멍 장작비 2인기준 2만5천원 내면 5키로 장작 1망 줌 (곰 팡이냄새나는 젖은 장작으로 줌)추가 1망당 1만2천원 *오로라가루 라면 스프만한 1봉지 주는데 그냥 불색깔마 초록색으로 잠깐 변함

4. 숯그릴 인원수별 금액이 차등 우린 4만5천원짜리 숮무한리필 선택했으나 처음에 숯2봉지 주고 추가로 달리고 했더니 무한리필이리고 4만 5천원짜리 팔고서는 말바꾸기 1봉지당 5천원추가로 구입해서 가저가라고 함

5. 이불, 비개, 침대침구류, 수건, 지하실어서 말린 곰팡이냄새가 남

6. 시설 노후 리뷰 사진처럼 바닥 조심 썩은 나무로 마루같은거 밟으면 다칠수있음 발이빠저 다쳐 넘어저 피가나고 멍이 생김 항의하자 사과없이 후시딘만 바르라고 가져다 주는것도 아니고 관리실로 오라함

7. 아이들 놀이터가있는데 방방이 찌져저있고 귀신나올것같은 놀이터

결론 여자사장은 친절한 척하지만 반말에 여러번 말해야 들어주고 손님 뚱뚱하니 등. 말 가리지 않고 하는 스타일 남자사장은 고객 무시하는 말투와 대충 말로 때우고 넘어가려함
시설 노후 냄새에 민감한 사람 조심하세요
진심으로 제 리뷰보고 저같은 피해보지 마세요'''):
    if t in SO:
        print(t, SO[t])
        sumSO += SO[t]
sumSO

In [99]:
reviews['clean4'] = reviews['clean2'].apply(lambda d:d.split())
reviews['clean5'] = reviews['clean3'].apply(lambda d:d.split())

In [103]:
K = 2   # 형용사
K = 4   # 명사
alpha = .1
beta = 1
theta = dict()  # 문서-토픽 분포(빈도) => m번째 문서, k번째 토픽 => {m:{k:빈도}}
phi = dict()  # 단어픽 분포(빈도) => m번째 문서, k번째 토픽 => {m:{k:빈도}}
z = dict()  # 문서-단어 토픽 할당 정보 => m번째 문서, l번째 단어 => {m:{l:k}}

V = list()

# corpus = reviews['clean4']
corpus = reviews['clean5']

for m, d in enumerate(corpus):
    for l, w in enumerate(d):
        V.append(w.lower())
        if m not in z:
            z[m] = dict()
        if l not in z[m]:
            z[m][l] = 0
        z[m][l] = randint(1, K)  # 토픽 할당

V = set(V)
i2w = {i: v for i, v in enumerate(V)}
w2i = {v: i for i, v in enumerate(V)}

In [104]:
topicWordsCount = dict() # 계산 빨리 하려고 토픽별 할당된 단어의 수

for m, L in z.items():
    if m not in theta:
        theta[m] = dict()
    for l, t in L.items():
        if t not in theta[m]:
            theta[m][t] = 0
        theta[m][t] += 1

        v = w2i[corpus[m][l].lower()]

        if v not in phi:
            phi[v] = dict()
        if t not in phi[v]:
            phi[v][t] = 0
        phi[v][t] += 1

        if t not in topicWordsCount:
            topicWordsCount[t] = 0
        topicWordsCount[t] += 1

In [105]:
iteration = 100

for _ in range(iteration):
    for m, d in enumerate(corpus):
        for l, w in enumerate(d):
            v = w2i[w.lower()]

            k = z[m][l] # gibb's 대상 삭제
            theta[m][k] -= 1
            phi[v][k] -= 1
            topicWordsCount[k] -= 1

            topic = list()
            for t in range(K):
                # A = (phi[v].get(t+1,0) + beta) / sum([T.get(t+1,0)+beta for v, T in phi.items()])
                A = (phi[v].get(t+1,0) + beta) / (topicWordsCount[t+1] * (beta+1))

                B = theta[m].get(t+1,0) + alpha
                topic.append(A*B)

            k = choices(range(K), weights=topic, k=1)[0]+1

            z[m][l] = k
            theta[m][k] = (theta[m][k] if k in theta[m] else 0) + 1
            phi[v][k] = (phi[v][k] if k in phi[v] else 0) + 1
            topicWordsCount[k] += 1

In [106]:
for k in range(K):
    result = {i2w[l]: T[k+1] for l, T in phi.items() if T.get(k+1,0) > 0}
    print(sorted(result.items(), key=lambda r:r[1], reverse=True)[:20])

[('화장실', 1107), ('글램', 560), ('카라', 558), ('이용', 466), ('친절', 439), ('시설', 426), ('사용', 389), ('침대', 363), ('사장', 338), ('샤워실', 319), ('생각', 316), ('텐트', 305), ('정도', 300), ('불편', 286), ('냄새', 275), ('내부', 273), ('청결', 273), ('바닥', 266), ('식기', 261), ('수건', 255)]
[('친절', 791), ('카라', 663), ('사장', 630), ('수영장', 545), ('시설', 508), ('이용', 465), ('아이들', 439), ('글램', 429), ('가족', 366), ('숙소', 362), ('다음', 360), ('방문', 340), ('캠핑', 324), ('계곡', 324), ('예약', 316), ('화장실', 298), ('생각', 291), ('여행', 288), ('만족', 244), ('아이', 240)]
[('사장', 358), ('글램', 328), ('예약', 308), ('시간', 308), ('친절', 297), ('사람', 232), ('시설', 222), ('추가', 221), ('리뷰', 203), ('기분', 202), ('생각', 199), ('처음', 186), ('소리', 183), ('만원', 179), ('사진', 163), ('직원', 162), ('새벽', 159), ('전화', 158), ('도착', 153), ('정도', 153)]
[('사장', 1956), ('친절', 1928), ('글램', 1167), ('시설', 909), ('방문', 686), ('다음', 673), ('!!', 642), ('화장실', 523), ('감사', 446), ('만족', 439), ('최고', 359), ('처음', 345), ('!!!', 322), ('고양이', 319), ('청결', 309), ('힐링', 30

In [107]:
sumSO = 0.0
for pTerms in [('아니', 541), ('나오', 411), ('그렇', 307), ('드리', 298), ('괜찮', 270), ('아쉽', 262), ('시끄럽', 235), ('못하', 230), ('챙기', 227), ('보이', 215), ('들어가', 215), ('다니', 206), ('모르', 189), ('편하', 170), ('이렇', 137), ('더럽', 133), ('힘들', 129), ('어떻', 120), ('가깝', 118), ('나쁘', 117)]:
    sumSO += SO[pTerms[0]]

sumSO

-16.083106740967033